In [1]:
import pandas as pd
import numpy as np
from numpy import average
from pandas_profiling import ProfileReport
import dtale

In [2]:
import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
import plotly.express as px

In [3]:
# %run ./formateo_variables.ipynb
# %run ./Tesis_estadarización_construcción_variables_2.ipynb
%run ./bases_datos_tesis.ipynb

## Análisis descriptivo

Está sección presenta estadísticas descriptivas de la muestra, asi como de la PET, la PEA, los ocupados, los desocupados y los inactivos, con el fin de contrastar el comportamiento general con los grupos de tratamiento y control. En este sentido, se compara el tiempo promedio de desempleo y de empleo por sexo, edad, posición ocupacional, rama de actividad, educación. Posteriormente se incluyen las variables dependientes de ingreso y calidad del empleo.


In [4]:
d = dtale.show(dfgroups, ignore_duplicate = True)
d

Executing shutdown due to inactivity...


2022-02-25 13:58:39,625 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-02-25 13:58:43,693 - INFO     - Executing shutdown...


In [138]:
# d = dtale.show(dfgroups, ignore_duplicate = True)
# d

In [11]:
#dfprueba.to_csv('D:/maestria_ciencias_economicas_unal/tesis_scarring/base datos/ELCA_FINAL/PET_REDUC_Trat_ctrl.csv', sep = ";")
# df7.to_csv('D:/maestria_ciencias_economicas_unal/tesis_scarring/base datos/ELCA_FINAL/PANEL_PET_REDUC.csv', sep = ";")

###  Indentificación de los grupos - pivot table

#### PET

In [6]:
#tabla 1 word
prueba2= df7.groupby(by=["Fecha","clasif_lab"]).aggregate({"edad": [np.size]})#, np.max, np.min, np.mean, np.std]})
prueba2

edad
                       size
Fecha      clasif_lab      
2010-12-31 1.0         4009
           2.0          554
           3.0         1270
2013-12-31 1.0         4172
           2.0          338
           3.0         1323
2016-12-31 1.0         4129
           2.0          269
           3.0         1391

In [10]:
df7["tiempo_desempleo"] = (df7["t_bustrab_a"]*12) + (df7["t_bustrab_m"])

#### tratamiento y control


In [24]:
#tabla 2
trat_control = dfprueba.pivot_table(values= ["id"],
                                  index=["Fecha"],#, "sexo"],
                                  columns=["scar"], aggfunc= "count", margins = True)
trat_control #iloc[:, [1,4]]

id            
scar         0.0   1.0   All
Fecha                       
2010-12-31  1098  1229  2327
2013-12-31  1098  1229  2327
2016-12-31  1098  1229  2327
All         3294  3687  6981

### PEA

#### Tiempo  empleo y desempleo continuos

In [18]:
#totales tiempo de empleo y desempleo actual. Expandidos
t1= round(dfgroups.groupby(["educación3", "scar"]).apply(
    lambda x: pd.Series(np.ma.average(
        np.ma.masked_array(x[["tiempo_empleo", "tiempo_desempleo"]], 
                           np.isnan(x[["tiempo_empleo", "tiempo_desempleo"]])), 
        weights=x.factorex_plong, axis=0),
                        index=["tiempo_empleo", "tiempo_desempleo"])).unstack(),2)
t1.iloc[[2,0,1,5,4,6,3],[1,3,0,2]]

,tiempo_empleo,tiempo_desempleo,tiempo_empleo,tiempo_desempleo
scar,1.0,1.0,0.0,0.0
educación3,,,,
Ninguno,38.28,17.99,46.08,19.54
Básica primaria (1 a 5),59.81,15.65,53.32,11.02
Básica secundaria y media (6 a 13),35.32,15.96,43.65,11.27
Técnico,25.28,12.57,38.33,16.63
Tecnológico,31.90,26.50,49.75,NaN
Universitario,40.77,37.92,38.89,9.04
Posgrado,25.35,20.04,54.41,NaN


In [30]:
#totales tiempo de empleo y desempleo actual por posición ocupacional. Expandidos
te= round(df8.groupby(["ocupacion2","Fecha"]).apply(
    lambda x: pd.Series(np.ma.average(
        np.ma.masked_array(x["tiempo_empleo"], 
                           np.isnan(x["tiempo_empleo"])), 
        weights=x.factorex_plong, axis=0),
                        index=["tiempo_empleo"])),2)
te#.iloc[40:60,:]

tiempo_empleo
ocupacion2                                         Fecha                    
Asalariado de empresa particular                   2010-12-31          91.84
                                                   2013-12-31          44.02
                                                   2016-12-31          53.23
Asalariado del gobierno                            2010-12-31         164.17
                                                   2013-12-31          63.04
                                                   2016-12-31          72.83
Empleado doméstico                                 2010-12-31          92.98
                                                   2013-12-31          34.40
                                                   2016-12-31          32.27
Jornalero o peón                                   2010-12-31          97.59
                                                   2013-12-31          66.62
                                                   2016-12-31          81.58
Otro                                               2010-12-31          53.79
                                                   2013-12-31          27.30
                                                   2016-12-31         152.77
Patrón o empleador                                 2010-12-31         130.38
                                                   2013-12-31          63.47
                                                   2016-12-31          57.49
Trabajador de su propia finca independientement... 2010-12-31         182.93
                                                   2013-12-31         101.15
                                                   2016-12-31         187.08
Trabajador familiar sin remuneración               2010-12-31          64.87
                                                   2013-12-31          26.84
                                                   2016-12-31          33.30
Trabajador por cuenta propia                       2010-12-31         130.00
                                                   2013-12-31          47.13
                                                   2016-12-31          59.49

#### Tiempo empleo y desempleo retrospectivo - General


In [210]:
tr= round(df8.groupby(["Fecha", "tiempo_desemp"]).apply(
    lambda x: pd.Series(np.ma.sum(np.ma.masked_array(x["factorex_plong"], np.isnan(x["factorex_plong"]))))),0)
tr

0
Fecha      tiempo_desemp                     
2013-12-31 Entre 1 año y 2 años      422678.0
           Entre 6 meses y 11 meses  405737.0
           Menos de 6 meses          808293.0
           Más de 2 años             283790.0
2016-12-31 Entre 1 año y 2 años      233500.0
           Entre 6 meses y 11 meses  269120.0
           Menos de 6 meses          458102.0
           Más de 2 años             258335.0

In [211]:
tr= round(df8.groupby(["Fecha", "tiempo_trab"]).apply(
    lambda x: pd.Series(np.ma.sum(np.ma.masked_array(x["factorex_plong"], np.isnan(x["factorex_plong"]))))),0)
tr

0
Fecha      tiempo_trab                       
2013-12-31 Entre 6 meses y 11 meses  197591.0
           Entre un (1) año y dos    429484.0
           Menos de seis (6) meses   181336.0
           Más de dos (2) años       871101.0
2016-12-31 Entre 1 año y 2 años      491357.0
           Entre 6 meses y 11 meses  190812.0
           Menos de 6 meses          276530.0
           Más de 2 años             970082.0

#### Tratamiento y Control

###### Tiempo Empleo y desempleo

In [207]:
t2= round(dfgroups.groupby(["Fecha","scar"]).apply(
    lambda x: pd.Series(np.ma.average(
        np.ma.masked_array(x[["tiempo_empleo", "tiempo_desempleo"]], 
                           np.isnan(x[["tiempo_empleo", "tiempo_desempleo"]])), 
        weights=x.factorex_plong, axis=0),
                        index=["tiempo_empleo", "tiempo_desempleo"])),2).unstack()
t2

tiempo_empleo        tiempo_desempleo       
scar                 0.0    1.0              0.0    1.0
Fecha                                                  
2010-12-31        116.99  67.98             9.66   8.56
2013-12-31         12.13  17.04              NaN  31.95
2016-12-31         14.11  32.88            17.71  18.30

In [29]:
# por olas y scar
scar_table_emp= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["scar"],
                                  columns="Fecha", aggfunc= ["mean"], margins = True)
round(scar_table_emp,2)#.iloc[:, [1,4]]

mean                             
      tiempo_empleo                             
Fecha    2010-12-31 2013-12-31 2016-12-31    All
scar                                            
0.0          110.90      12.07      13.85  43.62
1.0           74.40      18.69      37.24  43.90
All           91.38      15.32      26.22  43.76

In [7]:
#por sexo
table_2_sexo = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["sexo"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_sexo, 2).iloc[:, [1,0]]

mean       
       tiempo_empleo       
scar             1.0    0.0
sexo                       
hombre         51.02  53.16
mujer          29.82  34.45

In [6]:
#por rango edad
table_2_edad = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["rango_edad"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_edad,2).iloc[:, [1,0]]

mean       
           tiempo_empleo       
scar                 1.0    0.0
rango_edad                     
19-25              21.34  22.99
26-35              26.49  26.88
36-45              40.11  40.47
46-55              54.09  55.39
55+                67.34  45.93
< 19                0.00    NaN

In [7]:
#por estado civil
table_2_ec = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["estado_civil"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_ec,2).iloc[:, [1,0]]

mean       
                            tiempo_empleo       
scar                                  1.0    0.0
estado_civil                                    
Casado(a)                           43.75  47.58
En unión libre                      42.91  43.23
Separado(a) o divorciado(a)         30.38  33.83
Soltero(a)                          43.41  39.39
Viudo(a)                            46.98  35.42

In [8]:
#por estado parentesco
table_2_pa = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["parentesco"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_pa, 2).iloc[:, [1,0]]

mean       
                                           tiempo_empleo       
scar                                                 1.0    0.0
parentesco                                                     
Cónyuge o compañera(o)                             31.17  35.73
Hermano(a)                                         20.12   3.00
Hijastro(a)                                        41.92    NaN
Hijo(a)                                            16.50   7.67
Jefe(a)                                            45.53  47.89
Nieto(a)del jefe del hogar o de su cónyuge          8.50    NaN
Otro no pariente del jefe                           5.88    NaN
Padre o madre                                        NaN  12.33
Sobrino(a)                                         12.00    NaN
Suegro(a)                                         432.00    NaN
Yerno o nuera                                      11.62  30.00

In [9]:
#por estado etnia
table_2_et = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["etnia"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_et,2).iloc[:, [1,0]]

mean        
                                 tiempo_empleo        
scar                                       1.0     0.0
etnia                                                 
blanco                                   21.56   12.61
indígena                                 64.32   47.18
mestizo                                  16.50   12.20
negro, mulato (afrodescendiente)         48.12   73.55
ninguno de los anteriores                57.18   82.06
palenquero                                 NaN  132.50
rom o gitano                             42.73   72.00

In [13]:
#por estado estrato
table_2_est = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["estrato"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_est,2).iloc[:, [1,0]]

mean       
        tiempo_empleo       
scar              1.0    0.0
estrato                     
1.0             39.83  37.34
2.0             42.02  44.93
3.0             41.99  51.16
4.0             45.87  35.85

In [14]:
#por tipo vivienda
table_2_viv = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["tipo_vivienda"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_viv,2).iloc[:, [1,0]]

mean       
                                                   tiempo_empleo       
scar                                                         1.0    0.0
tipo_vivienda                                                          
Apartamento                                                37.20  41.08
Casa                                                       44.55  44.40
Cuarto                                                     27.40  50.88
Otro tipo de unidad de vivienda (carpa, tienda,...         14.33   0.00
Otro tipo de unidad de vivienda (carpa, tienda,...         37.25    NaN

In [12]:
#por conyugue
table_2_cy = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["vive_conyuge"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort =False)
round(table_2_cy,2).iloc[:, [1,0]]

mean       
             tiempo_empleo       
scar                   1.0    0.0
vive_conyuge                     
Si                   42.71  45.47
No                   93.43  53.13

In [16]:
#por lee y escribe
table_2_lee = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["lee_escribe"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(table_2_lee,2).iloc[:, [1,0]]

mean       
            tiempo_empleo       
scar                  1.0    0.0
lee_escribe                     
Si                  40.41  44.08
No                  62.90  34.50

In [18]:
# scar_table_empleo_año.compare(scar_table_empleo_año_n, keep_shape=True, keep_equal= False)

###### Tiempo desempleo

In [32]:
# (II) tiempo total de desempleo promedio. 
scar_table = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["scar"],
                                  columns="Fecha", aggfunc= ["mean"], margins = True)
#scar_table= round(scar_table,2).iloc[3:,:]
round(scar_table, 2)

mean                             
      tiempo_desempleo                             
Fecha       2010-12-31 2013-12-31 2016-12-31    All
scar                                               
0.0              12.80        NaN      13.85  13.03
1.0              11.51      29.08      13.30  21.01
All              11.93      29.08      13.44  19.63

In [49]:
#por sexo
table_2_sexo_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["sexo"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_sexo_d,2)

mean       
       tiempo_desempleo       
scar                0.0    1.0
sexo                          
hombre             8.34  22.45
mujer             17.72  20.23

In [52]:
#por rango edad
table_2_edad_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["rango_edad"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_edad_d,2)#, 2).iloc[:, [1,4,]]

mean       
           tiempo_desempleo       
scar                    0.0    1.0
rango_edad                        
19-25                  8.86  10.21
26-35                  9.13  22.02
36-45                 19.68  18.56
46-55                  2.89  25.92
55+                   19.62  23.26

D:\anaconda\lib\site-packages\dtale\views.py:1660: FutureWarning:

The default value of regex will change from True to False in a future version.



In [63]:
#por estado civil
table_2_ec_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["estado_civil"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_ec_d,2)#, 2).iloc[:, [1,4,]]

mean       
                            tiempo_desempleo       
scar                                     0.0    1.0
estado_civil                                       
Casado(a)                              16.14  28.09
En unión libre                          8.89  19.11
Separado(a) o divorciado(a)            22.80  13.76
Soltero(a)                             12.33  18.59
Viudo(a)                               13.00  30.83

In [64]:
#por parentesco
table_2_pa_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["parentesco"],
                                  columns=["scar"], aggfunc= ["count", "mean"], margins = False)
round(table_2_pa_d,2)#, 2).iloc[:, [1,4,]]

count         \
                                           tiempo_desempleo          
scar                                                    0.0    1.0   
parentesco                                                           
Cónyuge o compañera(o)                                 19.0  105.0   
Hermano(a)                                              0.0    1.0   
Hijastro(a)                                             NaN    0.0   
Hijo(a)                                                 1.0    7.0   
Jefe(a)                                                38.0  164.0   
Nieto(a)del jefe del hogar o de su cónyuge              NaN    0.0   
Otro no pariente del jefe                               NaN    0.0   
Padre o madre                                           0.0    NaN   
Sobrino(a)                                              NaN    0.0   
Suegro(a)                                               0.0    0.0   
Yerno o nuera                                           0.0    0.0   

                                                       mean         
                                           tiempo_desempleo         
scar                                                    0.0    1.0  
parentesco                                                          
Cónyuge o compañera(o)                                17.74  23.65  
Hermano(a)                                              NaN  36.00  
Hijastro(a)                                             NaN    NaN  
Hijo(a)                                               30.00   5.43  
Jefe(a)                                               10.24  19.89  
Nieto(a)del jefe del hogar o de su cónyuge              NaN    NaN  
Otro no pariente del jefe                               NaN    NaN  
Padre o madre                                           NaN    NaN  
Sobrino(a)                                              NaN    NaN  
Suegro(a)                                               NaN    NaN  
Yerno o nuera                                           NaN    NaN

In [71]:
#por  etnia
table_2_et_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["etnia"],
                                  columns=["scar"], aggfunc= ["count","mean"], margins = False)
round(table_2_et_d,2)#, 2).iloc[:, [1,4,]]

count                    mean  \
                                 tiempo_desempleo        tiempo_desempleo   
scar                                          0.0    1.0              0.0   
etnia                                                                       
blanco                                        0.0   25.0              NaN   
indígena                                      1.0    9.0             6.00   
mestizo                                       0.0   58.0              NaN   
negro, mulato (afrodescendiente)              7.0   15.0            12.14   
ninguno de los anteriores                    37.0  127.0            13.11   
palenquero                                    0.0    NaN              NaN   
rom o gitano                                  0.0    0.0              NaN   

                                         
                                         
scar                                1.0  
etnia                                    
blanco                            40.92  
indígena                          20.11  
mestizo                           26.05  
negro, mulato (afrodescendiente)  21.60  
ninguno de los anteriores         17.76  
palenquero                          NaN  
rom o gitano                        NaN

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2021-10-04 10:36:15,936 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2021-10-04 10:36:20,009 - INFO     - Executing shutdown...


In [75]:
#por  estrato
table_2_est_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["estrato"],
                                  columns=["scar"], aggfunc= ["count","mean"], margins = False)
round(table_2_est_d,2)#, 2).iloc[:, [1,4,]]

count                  mean       
        tiempo_desempleo      tiempo_desempleo       
scar                 0.0  1.0              0.0    1.0
estrato                                              
1.0                   25  105            12.84  21.65
2.0                   20  114            14.00  20.28
3.0                   11   51            13.64  20.49
4.0                    2    7             2.50  27.00

In [80]:
#por  tipo de vivienda
table_2_viv_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["tipo_vivienda"],
                                  columns=["scar"], aggfunc= ["count","mean"], margins = False)
round(table_2_viv_d,2)#, 2).iloc[:, [1,4,]]

count         \
                                                   tiempo_desempleo          
scar                                                            0.0    1.0   
tipo_vivienda                                                                
Apartamento                                                    14.0   84.0   
Casa                                                           43.0  174.0   
Cuarto                                                          1.0   18.0   
Otro tipo de unidad de vivienda (carpa, tienda,...              0.0    0.0   
Otro tipo de unidad de vivienda (carpa, tienda,...              NaN    1.0   

                                                               mean         
                                                   tiempo_desempleo         
scar                                                            0.0    1.0  
tipo_vivienda                                                               
Apartamento                                                    7.57  21.50  
Casa                                                          15.09  20.39  
Cuarto                                                         1.00  23.83  
Otro tipo de unidad de vivienda (carpa, tienda,...              NaN    NaN  
Otro tipo de unidad de vivienda (carpa, tienda,...              NaN  36.00

In [87]:
#por conyugue
table_2_cy_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["vive_conyuge"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False)
round(table_2_cy_d,2)#, 2).iloc[:, [1,4,]]

mean       
             tiempo_desempleo       
scar                      0.0    1.0
vive_conyuge                        
No                        NaN  12.50
Si                      12.06  22.99

In [93]:
#por lee y escribe
table_2_lee_d = dfprueba.pivot_table(values= ["tiempo_desempleo"],
                                  index=["lee_escribe"],
                                  columns=["scar"], aggfunc= ["count","mean"], margins = False)
round(table_2_lee_d,2)#, 2).iloc[:, [1,4,]]

count                  mean       
            tiempo_desempleo      tiempo_desempleo       
scar                     0.0  1.0              0.0    1.0
lee_escribe                                              
No                         2   12             1.50  21.42
Si                        56  265            13.45  20.99

#### Estadísticas Ctr. y Trat. T. Empl y Desemp.

In [17]:
#por estudia
table_2_estu = dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["estudia"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(table_2_estu,2).iloc[[1,0], [3,1,2,0]] #el signo (-) en iloc cuenta de iizquierda a derecha las columnas

mean                                                
        tiempo_empleo tiempo_desempleo tiempo_empleo tiempo_desempleo
scar              1.0              1.0           0.0              0.0
estudia                                                              
Si              27.35            24.75         36.95            12.00
No              42.15            20.65         43.88            13.05

In [18]:
#por nivel educativo
table_2_nedu = dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["educación"],
                                  columns=["scar"], aggfunc= ["count"], margins = False, sort = False)
round(table_2_nedu,2).iloc[:, [3,1,2,0]] 

count                   \
                                   tiempo_empleo tiempo_desempleo   
scar                                         1.0              1.0   
educación                                                           
Posgrado con título                           37                3   
Posgrado sin título                           20                5   
Básica secundaria y media (6 a 13)          1435              124   
Técnico sin título                            90               11   
Básica primaria (1 a 5)                      842               66   
Tecnológico con título                        87                7   
Universitario sin título                     157               19   
Universitario con título                     157               13   
Técnico con título                           237               20   
Tecnológico sin título                        37                2   
Ninguno                                      104                7   

                                                                   
                                   tiempo_empleo tiempo_desempleo  
scar                                         0.0              0.0  
educación                                                          
Posgrado con título                           83                0  
Posgrado sin título                           34                0  
Básica secundaria y media (6 a 13)          1256               35  
Técnico sin título                            76                2  
Básica primaria (1 a 5)                      816                9  
Tecnológico con título                        81                0  
Universitario sin título                     120                0  
Universitario con título                     155                4  
Técnico con título                           187                5  
Tecnológico sin título                        20                0  
Ninguno                                      148                3

In [ ]:
##educación (falta consolidar la variable df7 y dfprueba)

In [21]:
#ocupación
table_2_ocu = dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["ocupacion2"],
                                  columns=["scar"], aggfunc= ["count","mean"], margins = False, sort = False)
round(table_2_ocu,2).iloc[:, [5,4,3,2]] 

mean         \
                                                   tiempo_empleo          
scar                                                         1.0    0.0   
ocupacion2                                                                
Asalariado de empresa particular                           32.25  38.89   
Trabajador por cuenta propia                               50.51  45.50   
Empleado doméstico                                         18.50  28.06   
Otro                                                       38.98  43.71   
Patrón o empleador                                         94.03  62.18   
Trabajador de su propia finca independientement...        108.03  48.59   
Jornalero o peón                                           56.02  29.09   
Asalariado del gobierno                                    34.49  57.25   
Trabajador familiar sin remuneración                       19.98  35.42   

                                                           count        
                                                   tiempo_empleo        
scar                                                         1.0   0.0  
ocupacion2                                                              
Asalariado de empresa particular                            1321   916  
Trabajador por cuenta propia                                1295  1503  
Empleado doméstico                                           182   107  
Otro                                                          56    31  
Patrón o empleador                                            41    84  
Trabajador de su propia finca independientement...            28    22  
Jornalero o peón                                             141    46  
Asalariado del gobierno                                      104   200  
Trabajador familiar sin remuneración                          35    67

In [18]:
#anos totales estudios superiores
#ocupación
table_2_sup = dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["anos_superior2"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(table_2_sup,2).iloc[:, [3,2,1,0]]


mean                              
               tiempo_empleo        tiempo_desempleo      
scar                     1.0    0.0              1.0   0.0
anos_superior2                                            
0-1                    23.55  36.54            14.78  12.5
2-3                    29.02  66.26            21.24   4.5
4-5                    38.08  61.62            29.33  24.0
5+                     47.01  66.94            28.00   5.0

In [87]:
#descripcion_ciiu
print(t_ciiu.columns) #print(t_ciiu.index)

MultiIndex([('count', 'tiempo_empleo',   0.0),
            ('count', 'tiempo_empleo',   1.0),
            ('count', 'tiempo_empleo', 'All'),
            ( 'mean', 'tiempo_empleo',   0.0),
            ( 'mean', 'tiempo_empleo',   1.0),
            ( 'mean', 'tiempo_empleo', 'All')],
           names=[None, None, 'scar'])


In [24]:
t_ciiu = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["descripcion_ciiu"],
                                  columns=["scar"], aggfunc= ["count","mean"], observed =True, margins = True, sort = False)
t_ciiu = round(t_ciiu,2)
t_ciiu = t_ciiu.sort_values([('count', 'tiempo_empleo', 'All')],ascending =False)
t_ciiu.iloc[0:15,[4,3]]


mean       
                                                   tiempo_empleo       
scar                                                         1.0    0.0
descripcion_ciiu                                                       
All                                                        41.41  43.58
Comercio al por mayor y al por menor; reparació...         18.67  12.29
Industrias manufactureras                                  24.18  13.01
Otras actividades de servicios                             28.13  12.28
Alojamiento y servicios de comida                          15.95  11.96
Transporte y almacenamiento                                23.18  12.60
Construcción                                               40.27  15.42
Agricultura, ganadería, caza, silvicultura y pesca         39.78  13.73
Actividades de los hogares individuales en cali...         19.09   9.08
Educación                                                  21.94  17.15
Actividades de atención de la salud humana y de...         20.41  14.04
Hogares privados con servicio doméstico                    59.89  65.36
Actividades de servicios administrativos y de a...         31.38  14.08
Construcción de edificaciones para uso recidencial        125.41  93.32
Actividades profesionales, científicas y técnicas          21.25  14.11

In [127]:
#cuenta las personas con tiempo de empleo > 90 meses por cada uno de los años de educación superior
# t_sup = dfprueba.pivot_table(values= ["tiempo_empleo"],
#                                   index=["anos_superior"],
#                                   columns=["scar"], aggfunc= {"tiempo_empleo": lambda x: (x >5).count()}, observed =False, margins = False, sort = True)
# t_sup
#unstack().(level=0) vuelve los indices de las pivot table en columnas
#stack().to_frame() vuelve las columnas de las pivot table indices
 #CrossTab:
#a partir de una lista de valores o series, en este caso necesitamos por un lado la lista de valores de Year y por otro lado la lista de valores de Month
#como primer argumento ponemos la serie o array que deseamos como indice (index), luego la serie en la cual se basan las columnas (columns), y si no establecemos una aggfunc utiliza ‘size’ por defecto
#Si los argumentos son de este tipo, hay que tener en cuenta que al crear la tabla crosstab utiliza el indice para hacer la unión entre ambas series. 
#En cambio si los objetos utilizados para crearla\ 
#son arrays de valores, se basa en el orden de los mismos para realizar la unión

D:\anaconda\lib\site-packages\dtale\views.py:1660: FutureWarning:

The default value of regex will change from True to False in a future version.



In [144]:
#print(t_horas.columns)
print(dfprueba["horas_normal"].value_counts())

48.0     1506
60.0      411
40.0      396
50.0      239
30.0      238
         ... 
102.0       2
86.0        1
107.0       1
73.0        1
23.0        1
Name: horas_normal, Length: 99, dtype: int64q

In [28]:
#horas_normal_2
t_horas = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["horas_normal_2"],
                                  columns=["scar"], aggfunc= ["mean"], observed =True, margins = False, sort = True)
round(t_horas,2).iloc[:,[1,0]]
# t_horas = t_ciiu.sort_values([('count', 'tiempo_empleo', 'All')],ascending =False)
# t_ciiu.iloc[0:15,[4,3]]
#dfprueba["horas_normal_2"].value_counts()

mean       
               tiempo_empleo       
scar                     1.0    0.0
horas_normal_2                     
-25                    35.76  30.80
25-47                  44.34  46.47
48                     43.14  47.39
48+                    41.35  44.81

In [29]:
#tipo de contrato
t_contr = dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["tipo_contrato"],
                                  columns=["scar"], aggfunc= ["mean"], observed =True, margins = False, sort = False)
round(t_contr,2).iloc[:,[1,0]]

mean       
                                      tiempo_empleo       
scar                                            1.0    0.0
tipo_contrato                                             
Contrato escrito a término fijo               23.25  11.05
No tiene contrato                             26.90  12.72
Contrato verbal                               25.82  11.43
Contrato escrito a término indefinido         23.57  15.52
No aplica                                     20.06  11.81

In [204]:
#ingreso laboral, no laboral y total
ing_l = dfprueba.pivot_table(values= ["Ingreso_laboral", "Ingreso_no_laboral", "Ingreso_total"],
                                  index=["scar"],
                                  aggfunc= ["mean"], observed =True, margins = True, sort = False)
round(ing_l,2)#.iloc[:,[1,0]]

mean                                 
     Ingreso_laboral Ingreso_no_laboral Ingreso_total
scar                                                 
1.0        785518.80          571762.82     710556.79
0.0       1133637.81          772629.72     970525.63
All       1357025.28          540136.67    1897161.95

In [116]:
##quintiles Ingreso_laboral	Ingreso_no_laboral	Ingreso_total
#dfprueba["Ingreso_total"].quantile([0.1, 0.2,0.3,0.4,0.5])
quin_it= dfprueba.pivot_table(values= ["tiempo_empleo", "Ingreso_total"],
                                  index=["q_Ingreso_total"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(quin_it,2).iloc[:, [3,2,1,0]] 

mean                                 
                tiempo_empleo        Ingreso_total            
scar                      1.0    0.0           1.0         0.0
q_Ingreso_total                                               
1.0                     44.05  43.41     182973.45   184306.85
2.0                     54.52  53.54     437853.98   440710.14
3.0                     35.36  38.49     626486.19   626792.47
4.0                     37.37  44.93     866519.23   899129.20
5.0                     37.55  41.62    1960761.60  2225212.01

In [117]:
##quintiles Ingreso_laboral	Ingreso_no_laboral	Ingreso_total
#dfprueba["Ingreso_total"].quantile([0.1, 0.2,0.3,0.4,0.5])
quin_it= dfgroups.pivot_table(values= ["tiempo_empleo", "Ingreso_total_dflac"],
                                  index=["q_Ingreso_total"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(quin_it,2).iloc[:, [3,2,1,0]] 

mean                                       
                tiempo_empleo        Ingreso_total_dflac            
scar                      1.0    0.0                 1.0         0.0
q_Ingreso_total                                                     
1.0                     41.04  41.10           170505.15   164777.14
2.0                     55.81  53.67           426728.05   421931.50
3.0                     34.25  35.83           634956.31   634800.26
4.0                     38.56  47.33           873910.07   906711.10
5.0                     37.64  41.69          1993342.50  2257259.98

In [51]:
quin_il= dfprueba.pivot_table(values= ["tiempo_empleo", "Ingreso_laboral"],
                                  index=["q_Ingreso_laboral"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(quin_il,2).iloc[:, [3,2,1,0]] 

mean                                   
                  tiempo_empleo        Ingreso_laboral            
scar                        1.0    0.0             1.0         0.0
q_Ingreso_laboral                                                 
1.0                       34.75  27.12       252182.98   279463.51
2.0                       33.18  45.20       546446.92   551795.71
3.0                       29.88  31.35       717966.01   712161.38
4.0                       33.27  43.97       974772.90  1007027.09
5.0                       34.37  46.01      2109359.17  2277781.67

In [53]:
quin_inl= dfprueba.pivot_table(values= ["tiempo_empleo", "Ingreso_no_laboral"],
                                  index=["q_Ingreso_no_laboral"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(quin_inl,2).iloc[:, [3,2,1,0]]  

mean                                      
                     tiempo_empleo        Ingreso_no_laboral            
scar                           1.0    0.0                1.0         0.0
q_Ingreso_no_laboral                                                    
1.0                          41.74  45.65          118269.32   123899.87
2.0                          56.41  54.07          290541.67   293277.67
3.0                          64.22  53.38          499572.60   503506.84
4.0                          52.63  42.61          761659.39   758320.51
5.0                          45.57  39.55         1678811.42  1953111.16

##### Calidad del empleo

In [48]:
#cotización a salud: segsoc_slud 2013 y 2016 ocupados. Otra variable: afiliacion, todos los años independiente del estado laboral
salud= dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["segsoc_salud"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(salud,2).iloc[:, [1,0]]

mean       
             tiempo_empleo       
scar                   1.0    0.0
segsoc_salud                     
Sí                   22.98  14.46
No                   25.73  12.21

In [22]:
#cotización a pensión: afiliacion_fp 2013 y 2016 ocupados. Otra variable: cotiza_fp 2010 y cotizando 2013 y 2016 independiente del estado laboral
salud= dfprueba.pivot_table(values= ["tiempo_empleo", "tiempo_desempleo"],
                                  index=["afiliacion_fp"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(salud,2).iloc[:, [1,0]] 

mean       
              tiempo_empleo       
scar                    1.0    0.0
afiliacion_fp                     
Sí                    23.94  14.42
No                    25.23  12.28

In [74]:
#tamaño de la empresa
tam= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["ta_empresa"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(tam,2).iloc[:, [1,0]]


mean       
                    tiempo_empleo       
scar                          1.0    0.0
ta_empresa                              
50 personas y más           33.79  47.26
De 11 a 19 personas         32.59  35.53
De 2 a 5 personas           39.45  42.97
De 20 a 49 personas         28.78  41.14
De 6 a 10 personas          34.67  36.37
Trabaja solo                53.67  45.58

In [76]:
#registro mercantil de la empresa
reg= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["registro_mercantil", "Fecha"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = True)
round(reg,2).iloc[:, [1,0]]

mean        
                                                  tiempo_empleo        
scar                                                        1.0     0.0
registro_mercantil                     Fecha                           
Lo necesita pero no lo tiene           2010-12-31        105.97  119.12
                                       2013-12-31         12.94    9.95
No informa                             2010-12-31           NaN   60.00
No lo necesita                         2010-12-31        113.98  130.28
                                       2013-12-31         14.98   11.67
Si lo tiene pero no lo renovó este año 2010-12-31         75.17  126.97
Si lo tiene y lo renovó este año       2010-12-31         73.51  114.49
Sí lo tiene pero no lo renovó este año 2013-12-31         10.54   11.22
Sí lo tiene y lo renovó este año       2013-12-31         10.11   12.97

Executing shutdown due to inactivity...


2021-10-22 07:34:10,603 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-10-22 07:34:14,731 - INFO     - Executing shutdown...


In [25]:
#informalidad pensiones.
salud= dfprueba.pivot_table(values= ["id"],
                                  index=["cotiza_fp"],#afiliacion_fp
                                  columns=["scar"], aggfunc= ["count"], margins = False, sort = False)
round(salud,2)#.iloc[:, [1,0]] 

count      
                                                       id      
sexo                                               hombre mujer
cotiza_fp                                                      
No cotiza porque no tiene dinero                      990  1425
No cotiza porque no trabaja                           155  1005
Si está cotizando, pero todavía no es pensionado      862   615
Si está cotizando y recibe pensión                     15    19
No, porque ya está pensionado                         134    87
No cotiza porque no tiene la edad legal para ha...     25    25
No cotiza porque no quiere o no le interesa           219   221
No cotiza porque está esperando cumplir la edad...     19    17

In [6]:
#informalidad tamaño de la empresa
salud= dfprueba.pivot_table(values= ["id"],
                                  index=["ta_empresa", "Fecha"],
                                  columns=["scar"], aggfunc= ["count"], margins = False, sort = False)
round(salud,2)#.iloc[:, [1,0]] 

count     
                                  id     
scar                             0.0  1.0
ta_empresa          Fecha                
50 personas y más   2010-12-31   190  221
                    2013-12-31   209  191
                    2016-12-31   198  239
Trabaja solo        2010-12-31   382  372
                    2013-12-31   399  333
                    2016-12-31   426  407
De 11 a 19 personas 2010-12-31    38   44
                    2013-12-31    45   51
                    2016-12-31    37   56
De 2 a 5 personas   2010-12-31   248  286
                    2013-12-31   253  247
                    2016-12-31   218  259
De 20 a 49 personas 2010-12-31    41   72
                    2013-12-31    45   75
                    2016-12-31    52   80
De 6 a 10 personas  2010-12-31    47   85
                    2013-12-31    52   79
                    2016-12-31    58   84

In [10]:
#Tipo de empleo
t_em= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["Tipo_empleo"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(t_em,2).iloc[:, [1,0]] 

mean       
            tiempo_empleo       
scar                  1.0    0.0
Tipo_empleo                     
Formal              33.57  44.64
Informal            45.42  43.10

In [13]:
#Tipo de sector
t_se= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["tipo_sector"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(t_se,2).iloc[:, [1,0]] 

mean       
                tiempo_empleo       
scar                      1.0    0.0
tipo_sector                         
S. Formal               33.73  43.96
S. Informal             49.28  44.77
Hogar_empleador         35.48  30.95

In [16]:
#houssmans mix
t_hus= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["hussmanns"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(t_hus,2)#.iloc[:, [1,0]] 

mean       
                          tiempo_empleo       
scar                                0.0    1.0
hussmanns                                     
Ocupacion_formal                  44.99  33.68
Ocupacion_informal                44.95  50.04
Ocupacion_inf_fuera_s_inf         32.12  31.52
Ocupacion_en_s_inf                41.15  31.78

In [18]:
#houssmans a
t_hus_a= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["hussmanns_a"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(t_hus_a,2).iloc[:, [1,0]] 

mean       
                   tiempo_empleo       
scar                         1.0    0.0
hussmanns_a                            
Ocupacion_formal           33.68  44.99
Ocupacion_informal         44.58  43.01

In [20]:
#houssmans b
t_hus_b= dfprueba.pivot_table(values= ["tiempo_empleo"],
                                  index=["hussmanns_b"],
                                  columns=["scar"], aggfunc= ["mean"], margins = False, sort = False)
round(t_hus_b,2).iloc[:, [1,0]] 

mean       
                          tiempo_empleo       
scar                                1.0    0.0
hussmanns_b                                   
Ocupacion_en_s_inf                49.28  44.77
Ocupacion_inf_fuera_s_inf         31.52  32.12

In [285]:
# scar_table_1= dfprueba.groupby(by=["tiempo_desemp", "scar"]).aggregate({"edad": [np.size, np.max, np.min, np.mean, np.std],
#                                                                         "estrato": np.mean})
# scar_table_1.sort_values(["scar"], ascending = False)

#### Estadísticas descriptivas tabla 2

In [8]:
dfgroups.columns.to_list()

['Fecha',
 'ano_encuesta',
 'id',
 'ola_l',
 'hogar_numero',
 'orden',
 'seguimiento',
 'region_encuesta',
 'id_mpioU',
 'sexo',
 'edad',
 'rango_edad',
 'edad_meses',
 'parentesco',
 'estado_civil',
 'vive_conyuge',
 'orden_conyuge',
 'etnia',
 'estrato',
 'lee_escribe',
 'estudia',
 'educación',
 'educación2',
 'edad_dejoestudio',
 'anos_superior',
 'anos_superior2',
 'clasif_lab',
 'actividad_ppal',
 'diligencias4',
 'diligencias12',
 'actividad_principal',
 'ocupacion',
 'ocupacion2',
 'cod_oficio',
 'cod_oficio2',
 'descrip_oficio',
 'descripcion_ciiu',
 'horas_normal',
 'horas_normal_2',
 'tipo_contrato',
 'tiempo_empleo',
 'not_null_empl',
 'Ingreso_total',
 'Ingreso_laboral',
 'Ingreso_no_laboral',
 'q_Ingreso_total',
 'q_Ingreso_laboral',
 'q_Ingreso_no_laboral',
 'rec_otros_pago',
 'vr_alimentos_pago',
 'vr_vivienda_pago',
 'vr_educ_pago',
 'vr_subalimentacion',
 'vr_subtransporte',
 'vr_subfamiliar',
 'vr_bonos',
 'vr_ingmen_ase',
 'vr_salario',
 'vr_ganancia',
 'vr_ganancia

#### Cantidad de personas según scarring y valores expandidos 

In [128]:
#np.ma.:asifna a todos los datos una mascara de false, posteriormente con -->
#np.ma.masked: se asigna una mascara "true" de acuerdo con una especificación. 
#se excluyen estos valores "True" de cualquier calculo
# ejemplo:
# a = np.ma.arange(6).reshape((2, 3))
# a[1, :] = np.ma.masked
# a.sum()

masked

In [137]:
#valores para la variable clasificación laboral. Si se quita del eje me cuenta todos los valores 
#de la variable especificada reflejando el balanceo del panel:
# df8.groupby(["ano"], dropna=False).apply(
#     lambda x: pd.Series(np.ma.count(np.ma.masked_array(
#         x["clasif_lab"], np.isnan(x["clasif_lab"])))))
#con datos nulos para esa variable:
df8.groupby(["ano","clasif_lab"], dropna=False).agg({"clasif_lab":np.ma.count})

clasif_lab
ano    clasif_lab            
2010.0 1.0               4009
       2.0                554
       3.0               1270
       NaN                  4
2013.0 1.0               4172
       2.0                338
       3.0               1323
       NaN                  4
2016.0 1.0               4129
       2.0                269
       3.0               1391
       NaN                 48

In [71]:
###número de personas y valores expandidos por año
t1= round(dfgroups.groupby(["ano", "scar2"], dropna=False).apply(
    lambda x: pd.Series([np.ma.sum(np.ma.masked_array(
        x["factorex_plong"], np.isnan(x["factorex_plong"]))), 
                          x["id"].nunique()])),0).unstack().iloc[:,[0,2,1,3]]
t1

,0,1,0,1
scar2,0,0,1,1
ano,,,,
2010.0,3449730.0,2327.0,NaN,NaN
2013.0,2429682.0,1609.0,1020048.0,718.0
2016.0,1639257.0,1098.0,1810473.0,1229.0


In [7]:
#sumar con con condición de filtrado
dfgroups[dfgroups["ano"]==2010]["factorex_plong"].sum()

3449729.5945847984

In [30]:
#No. personas y valores expandidos por scar
l1=["sexo", "region_encuesta", "rango_edad", "estado_civil", "parentesco2","vive_conyuge",
    "tipo_vivienda","educación3", "descripcion_ciiu2", "rango_exper", "Tipo_empleo"]
#[dfgroups["ano"]==2016]
for i in l1:
    t1= round(dfgroups.groupby([i, "scar2"], dropna=False).apply(
    lambda x: pd.Series(np.ma.count(np.ma.masked_array(
        x["scar2"], np.isnan(x["scar2"]))))).unstack(),0)
#     t2= round(dfgroups.drop_duplicates(['id']).groupby([i, "scar2"], dropna=False).apply(
#     lambda x: pd.Series([x["id"].nunique(), np.ma.sum(x["factorex_plong"])])),0)
    t1["suma"] = t1[(0, 0)] + t1[(0, 1)]
    t1["sin_scar%"] = round((t1[(0, 0)] / t1[('suma', '')])*100,0)
    t1["con_scar%"] = round((t1[(0, 1)] / t1[('suma', '')])*100,0)
    print(pd.DataFrame(t1))
#print(t2)

           0        suma sin_scar% con_scar%
scar2      0     1                          
sexo                                        
hombre  2389  1061  3450      69.0      31.0
mujer   2645   886  3531      75.0      25.0
                    0       suma sin_scar% con_scar%
scar2               0    1                          
region_encuesta                                     
Atlántica        1400  484  1884      74.0      26.0
Bogotá            646  341   987      65.0      35.0
Central           891  276  1167      76.0      24.0
Oriental         1155  431  1586      73.0      27.0
Pacífica          942  415  1357      69.0      31.0
                 0           suma sin_scar% con_scar%
scar2            0      1                            
rango_edad                                           
19-25        254.0   37.0   291.0      87.0      13.0
26-35       1116.0  462.0  1578.0      71.0      29.0
36-45       1543.0  661.0  2204.0      70.0      30.0
46-55       1390.0  528.0  

In [8]:
#No. personas y valores expandidos por educación, sexo y scar
l1=["educación3"]
for i in l1:
    t1= round(dfgroups.groupby([i, "sexo", "scar2"], dropna=False).apply(
    lambda x: pd.Series(np.ma.count(np.ma.masked_array(
        x["scar2"], np.isnan(x["scar2"]))))).unstack(),0)
    print(t1)


                                              0     
scar2                                         0    1
educación3                         sexo             
Básica primaria (1 a 5)            hombre   728  318
                                   mujer    665  179
Básica secundaria y media (6 a 13) hombre   997  456
                                   mujer   1204  403
Ninguno                            hombre   124   57
                                   mujer     94   12
Posgrado                           hombre    73   21
                                   mujer     73   22
Tecnológico                        hombre    96   40
                                   mujer     77   29
Técnico                            hombre   165   78
                                   mujer    286  136
Universitario                      hombre   206   91
                                   mujer    246  105


In [28]:
t1.columns

MultiIndex([(     0,  0),
            (     0,  1),
            ('suma', '')],
           names=[None, 'scar2'])

In [36]:
#print(dfgroups.groupby(["ano"]).id.nunique())
#print(round(df8.groupby(["ano"]).agg({"factorex_plong":np.sum}),0))

In [62]:
#No. personas y valores expandidos PEA
l1=["sexo", "region_encuesta", "rango_edad", "estado_civil", "parentesco2","vive_conyuge",
    "tipo_vivienda","educación3", "descripcion_ciiu2", "rango_exper", "Tipo_empleo"]

for i in l1:
    t2= round(df8.groupby([i], dropna=False).apply(
            lambda x: pd.Series(np.ma.count(np.ma.masked_array(
                x["id"], np.isnan(x["id"]))))),0) #np.ma.sum(np.ma.masked_array(x["factorex_plong"], np.isnan(x["factorex_plong"])))
    t2["suma"] = t2[0].sum()
    t2["%"] = round((t2[0] / t2['suma'])*100,0)
#     t1["con_scar%"] = round((t1[(0, 1)] / t1[('suma', '')])*100,0)
    print(t2)
t2.columns

            0   suma     %
sexo                      
hombre   7257  17511  41.0
mujer   10254  17511  59.0
                       0   suma     %
region_encuesta                      
Atlántica           4401  17511  25.0
Bogotá              2609  17511  15.0
Central             3317  17511  19.0
Oriental            3594  17511  21.0
Orinoquía-Amazonía     2  17511   0.0
Pacífica            3588  17511  20.0
               0   suma     %
rango_edad                   
19-25        681  17511   4.0
26-35       3441  17511  20.0
36-45       4684  17511  27.0
46-55       4805  17511  27.0
55+         3876  17511  22.0
< 19          24  17511   0.0
                                0   suma     %
estado_civil                                  
Casado(a)                    6927  17511  40.0
En unión libre               6710  17511  38.0
Separado(a) o divorciado(a)  2147  17511  12.0
Soltero(a)                   1066  17511   6.0
Viudo(a)                      661  17511   4.0
                   

Index([0, 'suma', '%'], dtype='object')

In [61]:
t2[0]

Tipo_empleo
Formal      4579
Informal    7685
NaN         5247
Name: 0, dtype: int64

In [177]:
#ingreso total horario y horas promedio expandidos. PEA
msk_1 = df8["Ingreso_td_hora"].notna()
m=df8[msk_1==True]

t1= round(m.groupby(["ano"], dropna=True).apply(
        lambda x: pd.Series([np.ma.average(x["Ingreso_td_hora"],weights=x.factorex_plong, axis=0),
                             np.ma.count(x["Ingreso_td_hora"]), np.ma.sum(x["factorex_plong"])])),0)
print(t1)

              0       1          2
ano                               
2010.0  31100.0  3724.0  5685143.0
2013.0  33932.0  3904.0  5906315.0
2016.0  35066.0  3961.0  6015150.0


In [23]:
# #características personales Tiempo empleo y desempleo
# l1=["sexo", "rango_edad", "estado_civil", "parentesco", "etnia", "vive_conyuge", "estrato", 
#      "tipo_vivienda", "lee_escribe", "estudia", "educación", "anos_superior2"]

# for i in l1:
#     t1= round(dfgroups.groupby([i, "scar"]).apply(
#         lambda x: pd.Series(np.ma.average(
#             np.ma.masked_array(x[["tiempo_empleo", "tiempo_desempleo"]], 
#                                np.isnan(x[["tiempo_empleo", "tiempo_desempleo"]])), 
#             weights=x.factorex_plong, axis=0),
#                             index=["tiempo_empleo", "tiempo_desempleo"])).unstack(),2)
#     print(t1.iloc[:,[1,3,0,2]])

In [26]:
#características laborales
l1=["ocupacion2", "horas_normal_2", "tipo_contrato", "segsoc_salud", "afiliacion_fp", "ta_empresa", 
    "Tipo_empleo", "tipo_sector", "hussmanns", "hussmanns_a", "hussmanns_b"]

for i in l1:
    t1= round(dfgroups.groupby([i, "scar"]).apply(
        lambda x: pd.Series(np.ma.average(
            np.ma.masked_array(x["tiempo_empleo"], 
                               np.isnan(x["tiempo_empleo"])), 
            weights=x.factorex_plong, axis=0),
                            index=["tiempo_empleo"])).unstack(),2)
    print(t1.iloc[:,[1,0]])

                                                   tiempo_empleo       
scar                                                         1.0    0.0
ocupacion2                                                             
Asalariado de empresa particular                           36.45  45.81
Asalariado del gobierno                                    34.83  38.14
Empleado doméstico                                         19.58  67.42
Jornalero o peón                                           47.14  31.51
Otro                                                       26.84  47.14
Patrón o empleador                                         95.44  49.86
Trabajador de su propia finca independientement...         98.98  59.17
Trabajador familiar sin remuneración                       23.11  27.81
Trabajador por cuenta propia                               44.59  46.31
               tiempo_empleo       
scar                     1.0    0.0
horas_normal_2                     
-25                    28.98

In [4]:
#características laborales ingreso deflactado
l1=["Ingreso_total_dflac", "Ingreso_laboral_dflac", "Ingreso_no_laboral_dflac"]

for i in l1:
    t1= round(dfgroups.groupby(["scar"]).apply(
        lambda x: pd.Series(np.ma.average(
            np.ma.masked_array(x[[i]], 
                               np.isnan(x[[i]])), 
            weights=x.factorex_plong, axis=0),
                            index=[i])),2)
    print(t1.sort_values(by="scar",ascending=False))

      Ingreso_total_dflac
scar                     
1.0             936102.55
0.0            1218575.03
      Ingreso_laboral_dflac
scar                       
1.0              1074787.59
0.0              1373630.27
      Ingreso_no_laboral_dflac
scar                          
1.0                  676134.37
0.0                  956547.53


In [4]:
#ingreso total horario y horas promedio expandidos. 
msk_1 = dfgroups["Ingreso_td_hora"].notna()
m=dfgroups[msk_1==True]
l1=["Ingreso_td_hora", "horas_normal"]
for i in l1:
    t1= round(m.groupby(["scar2"]).apply(
         lambda x: pd.Series([np.ma.average(np.ma.masked_array(x[[i]], np.isnan(x[[i]])),
                                           weights=x.factorex_plong, axis=0),
                             np.ma.count(np.ma.masked_array(x[[i]], np.isnan(x[[i]]))),
                            np.ma.sum(np.ma.masked_array(x[["factorex_plong"]], 
                                                         np.isnan(x[["factorex_plong"]])))])),0)
    print(t1)

                         0     1          2
scar2                                      
0      [7271.782181910341]  4074  6141824.0
1      [6381.942718587623]  1796  2656383.0
                         0     1          2
scar2                                      
0      [48.77457690810528]  4074  6141824.0
1      [45.06327905053194]  1796  2656383.0


In [5]:
#características laborales quintiles ingreso deflactados y tiempo empleo
l1=["q_Ingreso_total", "q_Ingreso_laboral", "q_Ingreso_no_laboral"]
#l2 = ["tiempo_empleo","Ingreso_total", "Ingreso_laboral", "Ingreso_no_laboral"]
cols = [[1,3,0,2],[1,5,0,4], [1,7,0,6]]
for i,j in zip(l1,cols):
    t1= round(dfgroups.groupby([i, "scar"]).apply(
        lambda x: pd.Series(np.ma.average(
            np.ma.masked_array(x[["tiempo_empleo","Ingreso_total_dflac", "Ingreso_laboral_dflac", "Ingreso_no_laboral_dflac"]], 
                               np.isnan(x[["tiempo_empleo","Ingreso_total_dflac", "Ingreso_laboral_dflac", "Ingreso_no_laboral_dflac"]])), 
            weights=x.factorex_plong, axis=0),
                            index=["tiempo_empleo","Ingreso_total_dflac", "Ingreso_laboral_dflac", "Ingreso_no_laboral_dflac"])).unstack(),2)
    print(t1.iloc[:,j])

                tiempo_empleo Ingreso_total_dflac tiempo_empleo  \
scar                      1.0                 1.0           0.0   
q_Ingreso_total                                                   
1.0                     39.18           191742.92         43.29   
2.0                     50.34           491696.20         65.46   
3.0                     33.97           705149.55         41.66   
4.0                     32.39           975979.16         50.86   
5.0                     43.65          2366259.38         38.73   

                Ingreso_total_dflac  
scar                            0.0  
q_Ingreso_total                      
1.0                       182871.91  
2.0                       484622.58  
3.0                       704748.41  
4.0                      1023855.45  
5.0                      2471228.07  
                  tiempo_empleo Ingreso_laboral_dflac tiempo_empleo  \
scar                        1.0                   1.0           0.0   
q_Ingreso_laboral

In [39]:
#características laborales descripción ciuu
t1= round(dfgroups.groupby(["descripcion_ciiu", "scar"]).agg({"tiempo_empleo": [np.ma.count, np.mean]}).unstack(),2).sort_values(by=('tiempo_empleo', 'count',0), ascending = False)
print(t1.iloc[:10,:])
# t_ciiu = t_ciiu.sort_values([('count', 'tiempo_empleo', 'All')],ascending =False)
# t_ciiu.iloc[0:15,[4,3]]
      

                                                   tiempo_empleo         \
                                                           count          
scar                                                         0.0    1.0   
descripcion_ciiu                                                          
Comercio al por mayor y al por menor; reparació...         574.0  369.0   
Industrias manufactureras                                  259.0  257.0   
Otras actividades de servicios                             225.0  214.0   
Alojamiento y servicios de comida                          162.0  186.0   
Transporte y almacenamiento                                152.0  173.0   
Educación                                                  120.0   72.0   
Agricultura, ganadería, caza, silvicultura y pesca         100.0  161.0   
Construcción                                                92.0  211.0   
Actividades de atención de la salud humana y de...          76.0   77.0   
Actividades de los hogare

In [6]:
#características laborales descripción ciuu
t1= round(dfgroups.groupby(["descripcion_ciiu", "scar"]).apply(
    lambda x: pd.Series([np.ma.average(
        np.ma.masked_array(x["tiempo_empleo"], 
                           np.isnan(x["tiempo_empleo"])), 
        weights=x.factorex_plong, axis=0), np.ma.count(x["tiempo_empleo"])])
),2).unstack().sort_values(by=(1,0), ascending = False).iloc[:10,[1,0,3,2]]
         
print(t1) 

                                                        0             1       
scar                                                  1.0    0.0    1.0    0.0
descripcion_ciiu                                                              
Comercio al por mayor y al por menor; reparació...  15.15  13.25  369.0  574.0
Industrias manufactureras                           24.15  13.55  257.0  259.0
Otras actividades de servicios                      54.24  12.54  214.0  225.0
Alojamiento y servicios de comida                   14.53  10.90  186.0  162.0
Transporte y almacenamiento                         19.63  12.60  173.0  152.0
Educación                                           18.16  17.28   72.0  120.0
Agricultura, ganadería, caza, silvicultura y pesca  28.33  17.23  161.0  100.0
Construcción                                        36.78  12.54  211.0   92.0
Actividades de atención de la salud humana y de...  24.76  11.48   77.0   76.0
Actividades de los hogares individuales en cali...  

In [9]:
print(t1.index[:])

Index(['Comercio al por mayor y al por menor; reparación de vehículos automotores y moto',
       'Industrias manufactureras', 'Otras actividades de servicios',
       'Alojamiento y servicios de comida', 'Transporte y almacenamiento',
       'Educación', 'Agricultura, ganadería, caza, silvicultura y pesca',
       'Construcción',
       'Actividades de atención de la salud humana y de asistencia social',
       'Actividades de los hogares individuales en calidad de empleadores;actividades no'],
      dtype='object', name='descripcion_ciiu')


#### Estadísticas descriptivas según remuneración total horaria: trat, control y PEA (Tabla 3)

In [8]:
#características personales,socioeconómicas, laborales, remuneraciones y calidad del empleo
#por ingreso total horario deflactado. (Variables del modelo 1.)
l1=["sexo", "region_encuesta", "rango_edad", "estado_civil", "parentesco2","vive_conyuge",
    "tipo_vivienda","educación3", "descripcion_ciiu2", "rango_exper", "Tipo_empleo"]

for i in l1:
    t1= round(dfgroups.groupby([i, "scar2"]).apply(
        lambda x: pd.Series(np.ma.average(
            np.ma.masked_array(x["Ingreso_td_hora"], 
                               np.isnan(x["Ingreso_td_hora"])), 
            weights=x.factorex_plong, axis=0),
                            index=["Ingreso_td_hora"])).unstack(),0)
#     t2= round(df8.groupby(i).apply(
#         lambda x: pd.Series(np.ma.average(
#             np.ma.masked_array(x["Ingreso_td_hora"], 
#                                np.isnan(x["Ingreso_td_hora"])), 
#             weights=x.factorex_plong, axis=0),
#                             index=["Ingreso_td_hora"])),2)
    print(t1.iloc[:,[0,1]])
#     print(t2)

       Ingreso_td_hora        
scar2                0       1
sexo                          
hombre          7232.0  6470.0
mujer           7323.0  6280.0
                Ingreso_td_hora        
scar2                         0       1
region_encuesta                        
Atlántica                5809.0  5202.0
Bogotá                  10887.0  9215.0
Central                  5888.0  5750.0
Oriental                 6979.0  6212.0
Pacífica                 6514.0  4394.0
           Ingreso_td_hora        
scar2                    0       1
rango_edad                        
19-25               4208.0  3796.0
26-35               6445.0  5717.0
36-45               8517.0  6999.0
46-55               6192.0  6295.0
55+                 8828.0  6412.0
< 19                 577.0     NaN
                            Ingreso_td_hora        
scar2                                     0       1
estado_civil                                       
Casado(a)                            8107.0  7169.0
En

In [20]:
dfgroups["descripcion_ciiu2"].value_counts()

otra                             2506
Comercio; rep. de vehículos       943
Industrias manufactureras         516
Otras act. de servicios           439
Alojamiento y serv. de comida     348
Transporte y almacenamiento       325
Construcción                      303
Agri. g. caza silv. y pesca       261
Act. hogares empleadores          208
Educación                         192
Act. salud y asis. social         153
Name: descripcion_ciiu2, dtype: int64

In [217]:
#descripción ciuu según ingreso total deflactado
t1= dfgroups.groupby(["descripcion_ciiu", "scar"]).apply(
    lambda x: pd.Series([np.ma.average(
        np.ma.masked_array(x["Ingreso_total_dflac"], 
                           np.isnan(x["Ingreso_total_dflac"])), 
        weights=x.factorex_plong, axis=0), np.ma.count(x["Ingreso_total_dflac"])])
).unstack().sort_values(by=(1,0), ascending = False).iloc[:10,[1,0]]
t2= df8.groupby(["descripcion_ciiu"]).apply(
    lambda x: pd.Series([np.ma.average(
        np.ma.masked_array(x["Ingreso_total_dflac"], 
                           np.isnan(x["Ingreso_total_dflac"])), 
        weights=x.factorex_plong, axis=0), np.ma.count(x["Ingreso_total_dflac"])])
).sort_values(by=(1), ascending = False).iloc[:10,[0]]
         
print(t1.astype('float64').round(2))
print(t2.astype('float64').round(2))
#t3= t1.loc[:,[(0,0)]].round(2)
# t3

                                                             0            
scar                                                       1.0         0.0
descripcion_ciiu                                                          
Comercio al por mayor y al por menor; reparació...   817985.17   979023.35
Industrias manufactureras                            800807.43   999945.12
Otras actividades de servicios                       729074.70   789770.53
Alojamiento y servicios de comida                    802810.06   828905.70
Transporte y almacenamiento                         1052225.35  1412669.24
Educación                                           1008846.72  2230195.08
Agricultura, ganadería, caza, silvicultura y pesca   593092.84   622173.53
Construcción                                         836196.69  1458688.35
Actividades de atención de la salud humana y de...  1104391.95  1317448.62
Actividades de los hogares individuales en cali...   508691.60   384133.67
                         

In [213]:
t1.columns

MultiIndex([(0, 1.0),
            (0, 0.0)],
           names=[None, 'scar'])

##### Tiempo de desempleo y empleo retrospectivo

In [192]:
tr= round(dfgroups.groupby(["Fecha", "tiempo_desemp","scar"]).apply(
    lambda x: pd.Series(np.ma.sum(np.ma.masked_array(x["factorex_plong"], np.isnan(x["factorex_plong"]))))),0).unstack()
tr

0
scar                                      1.0
Fecha      tiempo_desemp                     
2013-12-31 Entre 1 año y 2 años      275159.0
           Entre 6 meses y 11 meses  271862.0
           Menos de 6 meses          641666.0
           Más de 2 años             171491.0
2016-12-31 Entre 1 año y 2 años      117405.0
           Entre 6 meses y 11 meses  208629.0
           Menos de 6 meses          355847.0
           Más de 2 años              53832.0

In [194]:
tr= round(dfgroups.groupby(["Fecha", "tiempo_trab","scar"]).apply(
    lambda x: pd.Series(np.ma.sum(np.ma.masked_array(x["factorex_plong"], np.isnan(x["factorex_plong"]))))),0).unstack()
tr.iloc[:,[1,0]]

0          
scar                                      1.0       0.0
Fecha      tiempo_trab                                 
2013-12-31 Entre 6 meses y 11 meses  126395.0    4345.0
           Entre un (1) año y dos    289775.0   24953.0
           Menos de seis (6) meses   123795.0   12268.0
           Más de dos (2) años       406944.0  106658.0
2016-12-31 Entre 1 año y 2 años      264024.0   20647.0
           Entre 6 meses y 11 meses  103821.0    5787.0
           Menos de 6 meses          121360.0   15240.0
           Más de 2 años             488667.0   95548.0

In [57]:
# (VII) excel. Tiempo de desempleo retrospectivo
scar_table_2 = dfprueba.pivot_table(values= ["id"],
                                  index=["Fecha","tiempo_desemp"],
                                  columns=["scar", "sexo"], aggfunc= "count", margins = True)
scar_table_total = scar_table_2#.iloc[:, [2,3]]
scar_table_total

id            
scar                                     1         All
sexo                                hombre mujer      
Fecha      tiempo_desemp                              
2013-12-31 Entre 6 meses y 11 meses    112    91   203
           Entre un (1) año y dos       79   110   189
           Menos de seis (6) meses     254   167   421
           Más de dos (2) años          46    70   116
2016-12-31 Entre 1 año y 2 años         36    53    89
           Entre 6 meses y 11 meses     56    69   125
           Menos de 6 meses            129    97   226
           Más de 2 años                18    34    52
All                                    730   691  1421

In [58]:
# (VIII) excel. tiempo de empleo retrospectivo
trat_empl = dfprueba.pivot_table(values= ["id"],
                                  index=["Fecha","tiempo_trab"],
                                  columns=["scar", "sexo"], aggfunc= "count", margins = True)
trat_empl = trat_empl.iloc[:, [2,3]]
trat_empl 

id      
scar                                     1      
sexo                                hombre mujer
Fecha      tiempo_trab                          
2013-12-31 Entre 6 meses y 11 meses     27    41
           Entre un (1) año y dos       72    98
           Menos de seis (6) meses      21    45
           Más de dos (2) años         173   148
2016-12-31 Entre 1 año y 2 años         69    88
           Entre 6 meses y 11 meses     23    45
           Menos de 6 meses             27    48
           Más de 2 años               195   159
All                                    607   672

### Factor de expansión para las estadisticas descriptivas

In [ ]:
#forma1

In [167]:
def w_av_ig_nan(df, col_value, col_weight):
    den = 0
    num = 0
    for index, row in df.iterrows():
        if (~np.isnan(row[col_weight]) & ~np.isnan(row[col_value])):
            den = den + row[col_weight]
            num = num + row[col_weight] * row[col_value]
        return num / den

In [212]:
w_av_ig_nan(dfgroups,"tiempo_empleo","factorex_plong")

36.0

In [ ]:
#forma 2 y 3
#2

In [226]:
x = np.array(dfgroups["tiempo_empleo"])
mx= np.ma.masked_array(x, np.isnan(x))

In [228]:
w= np.array(dfgroups["factorex_plong"])
np.ma.average(mx, axis=0, weights=w)

42.83970344724937

In [223]:
np.ma.average(np.ma.masked_array(dfgroups["tiempo_empleo"], np.isnan(dfgroups["tiempo_empleo"])),weights=dfgroups["factorex_plong"], axis=0)

42.83970344724937

In [252]:
wa_func =lambda x: np.average(np.ma.masked_array(x, np.isnan(x)), 
                              weights=dfgroups.loc[x.index, "factorex_plong"])

df1 = dfgroups.pivot_table(values=["tiempo_empleo"],
                     index=["sexo"],
                     columns=["scar"],
                     aggfunc={"tiempo_empleo": wa_func},
                     margins=False)

In [253]:
df1

tiempo_empleo           
scar             0.0        1.0
sexo                           
hombre     50.381536  42.775363
mujer      31.759270  25.392587

In [ ]:
#3

In [213]:
indices = np.where(np.logical_not(np.isnan(x)))[0]
np.average(x[indices], weights=w[indices])

42.839703447249384

In [ ]:
#forma 4

In [134]:
x = average(dfgroups["tiempo_empleo"][~np.isnan(dfgroups["tiempo_empleo"])])

In [77]:
var=dfgroups["tiempo_empleo"]
wgt=dfgroups["factorex_plong"]

In [214]:
indices = np.where(np.logical_not(np.isnan(var)))[0]
indices
np.average(var[indices], weights=wgt[indices])

nan

In [ ]:
#forma 5

In [193]:
def weighted_average(var, wgt):
    return round(sum([var[i]*wgt[i] for i in range(len(var))])/sum(wgt),2)

weighted_average(var,wgt)

nan

In [268]:
#stata table sexo scar [pw=factorex_plong], contents(mean  tiempo_empleo)
# dfgroups.groupby(["sexo", "scar"]).apply(lambda x : pd.Series(
#     {'tiempo_empleo': np.average(x.tiempo_empleo, weights=x.factorex_plong)}))
dfgroups.groupby(["sexo", "scar"]).apply(
    lambda x: pd.Series(np.ma.average(
        np.ma.masked_array(x.tiempo_empleo, np.isnan(x.tiempo_empleo)), 
        weights=x.factorex_plong),index=["tiempo_empleo"])).unstack()

tiempo_empleo           
scar             0.0        1.0
sexo                           
hombre     53.230169  47.986587
mujer      36.734533  30.321628

In [41]:
#dfgroups[((dfgroups["factorex_plong"].mul(dfgroups["tiempo_empleo"])).sum(min_count=1)) / ((dfgroups["factorex_plong"]).sum(min_count=1))]

In [74]:
print(dfgroups["factor_t_emp"][(dfgroups["sexo"]=="hombre") & ((dfgroups["scar"]==1))].sum(min_count=1)/a) 

42.77651365623852


In [99]:
dfgroups["factor_t_emp"] = dfgroups["factorex_plong"].mul(dfgroups["tiempo_empleo"])
table_2_sexo = dfgroups.pivot_table(values= ["factor_t_emp", "factorex_plong"], index=["sexo"],
                                    columns=["scar"],
                                    aggfunc= ["sum"])
table_2_sexo
#round(table_2_sexo / dfgroups["factorex_plong"].sum(min_count=1)) "factorex_plong"

sum                                           
        factor_t_emp               factorex_plong              
scar             0.0           1.0            0.0           1.0
sexo                                                           
hombre  1.371703e+08  1.184908e+08   2.722632e+06  2.770000e+06
mujer   6.971576e+07  6.757567e+07   2.195138e+06  2.661249e+06

In [255]:
table_2_sexo = dfgroups.pivot_table(values= ["factor_t_emp", "factorex_plong"], index=["sexo"],
                                    columns=["scar"],
                                    aggfunc= {"factor_t_emp":[np.sum], "factorex_plong": [lambda x: x.sum(min_count=1)]})
table_2_sexo

factor_t_emp               factorex_plong              
                 sum                     <lambda>              
scar             0.0           1.0            0.0           1.0
sexo                                                           
hombre  1.371704e+08  1.184950e+08   2.722632e+06  2.770170e+06
mujer   6.971598e+07  6.757601e+07   2.195138e+06  2.661249e+06

In [257]:
table_2_sexo1 = table_2_sexo[("factor_t_emp","sum")].divide(table_2_sexo[('factorex_plong','<lambda>')], axis = 0)
table_2_sexo1

scar,0.0,1.0
sexo,,
hombre,50.381536,42.775363
mujer,31.759270,25.392587


In [121]:
print(dfgroups["factorex_plong"][(dfgroups["factor_t_emp"]!=-1)].sum(min_count=1))

9172567.978233596


In [19]:
print(np.sum((dfgroups["factorex_plong"]) * (dfgroups["tiempo_empleo"]) / np.sum(dfgroups["factorex_plong"]))

37.97010052966295
